In [97]:
import numpy as np
from numba import cuda, jit, int32, float32, int64
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from math import pow, hypot, ceil
import random
import sys

## Read The problem data file:

In [2]:
class vrp():
    def __init__(self, capacity=None):
        self.capacity = capacity
        self.nodes = np.zeros((1,4), dtype=np.float32)
    def addNode(self, label, demand, posX, posY):
        newrow = np.array([label, demand, posX, posY], dtype=np.float32)
        self.nodes = np.vstack((self.nodes, newrow))

# Read the problem data file
def readInput():
	# Create VRP object:
    vrpManager = vrp()
	## First reading the VRP from the input ##
    print('Reading data file...', end=' ')
    fo = open('/home/conda_user/GA_VRP/test_set/P/P-n16-k8.vrp',"r")
    lines = fo.readlines()
    for i, line in enumerate(lines):
        while line.upper().startswith('CAPACITY'):
            inputs = line.split()
            vrpManager.capacity = np.float32(inputs[2])
			# Validating positive non-zero capacity
            if vrpManager.capacity <= 0:
                print(sys.stderr, 'Invalid input: capacity must be neither negative nor zero!')
                exit(1)
            break       
        while line.upper().startswith('NODE_COORD_SECTION'):
            i += 1
            line = lines[i]
            while not (line.upper().startswith('DEMAND_SECTION') or line=='\n'):
                inputs = line.split()
                vrpManager.addNode(np.int16(inputs[0]), 0.0, np.float32(inputs[1]), np.float32((inputs[2])))
                # print(vrpManager.nodes)
                i += 1
                line = lines[i]
                while (line=='\n'):
                    i += 1
                    line = lines[i]
                    if line.upper().startswith('DEMAND_SECTION'): break 
                if line.upper().startswith('DEMAND_SECTION'):
                    i += 1
                    line = lines[i] 
                    while not (line.upper().startswith('DEPOT_SECTION')):                  
                        inputs = line.split()
						# Validating demand not greater than capacity
                        if float(inputs[1]) > vrpManager.capacity:
                            print(sys.stderr,
							'Invalid input: the demand of the node %s is greater than the vehicle capacity!' % vrpManager.nodes[0])
                            exit(1)
                        if float(inputs[1]) < 0:
                            print(sys.stderr,
                            'Invalid input: the demand of the node %s cannot be negative!' % vrpManager.nodes[0])
                            exit(1)                            
                        vrpManager.nodes[int(inputs[0])][1] =  float(inputs[1])
                        i += 1
                        line = lines[i]
                        while (line=='\n'):
                            i += 1
                            line = lines[i]
                            if line.upper().startswith('DEPOT_SECTION'): break
                        if line.upper().startswith('DEPOT_SECTION'):
                            vrpManager.nodes = np.delete(vrpManager.nodes, 0, 0)                          
                            print('Done.')
                            return(vrpManager.capacity, vrpManager.nodes)

## Calculate fitness:

### CPU version

In [ ]:
# define fitness kernel here:
# @jit(nopython=True)
def fitness(cost_table, individual):
    zero_arr = np.zeros(1, dtype=np.int32)
    zeroed_indiv = np.copy(individual)
    
    # nodes represent the row/column index in the cost table
    for i in range(len(zeroed_indiv)):
        zeroed_indiv[i] = zeroed_indiv[i] - 1
        
    if zeroed_indiv[0] != 0:
        zeroed_indiv = np.hstack((zero_arr, zeroed_indiv))
    if individual[-1] != 1:
        zeroed_indiv = np.hstack((zeroed_indiv, zero_arr))
        
    fitness_val = 0
    for i in range(len(zeroed_indiv)-1):
        fitness_val += cost_table[int(zeroed_indiv[i]), int(zeroed_indiv[i+1])]
        
    return(fitness_val)

### GPU version

In [ ]:
# define fitness kernel here:
@cuda.jit
def fitness_gpu(cost_table_d, individual_d, zeroed_indiv_d, fitness_val_d):     
    
    # nodes represent the row/column index in the cost table
    threadId_row, threadId_col = cuda.grid(2)
    
    # Mapping between the 2D-grid indexing and the 1D-vector indexing:
    index = threadId_row*(cuda.blockDim.x)+threadId_col
    
    fitness_val_d[0] = 0
    if index+1 <= len(individual_d):
        zeroed_indiv_d[index] = individual_d[index] - 1
    
    if index == 0 and zeroed_indiv_d[index] != 0:
        cuda.atomic.add(fitness_val_d,0,cost_table_d[0, zeroed_indiv_d[index]])
        cuda.atomic.add(fitness_val_d,0,cost_table_d[zeroed_indiv_d[index], zeroed_indiv_d[index+1]])
    elif index == len(zeroed_indiv_d)-1 and zeroed_indiv_d[index] != 0:
        cuda.atomic.add(fitness_val_d,0,cost_table_d[zeroed_indiv_d[index], 0])
    elif index == len(zeroed_indiv_d)-1 and zeroed_indiv_d[index] == 0:
        pass
    elif index+1 <= len(zeroed_indiv_d):
        cuda.atomic.add(fitness_val_d,0,cost_table_d[zeroed_indiv_d[index], zeroed_indiv_d[index+1]])

In [ ]:
# Every individual MUST be initialized with length 2 * no._of_nodes

individual = np.array([8,15,1,5,12,1,14,9,1,11,16,16,1,6,6,4,1,2,1,3,1,7,1,\
                      8,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], dtype=np.int32)
# individual = np.array(range(10000), dtype=np.int32)
individual_d = cuda.to_device(individual)
zeroed_indiv_d = cuda.to_device(individual)

fitness_val_d = cuda.to_device(np.array([0], dtype=np.int32))

fitness_gpu[blocks,threads_per_block](cost_table_d, individual_d, zeroed_indiv_d, fitness_val_d)
print(fitness_val_d.copy_to_host()[0])

###############################################################################################
# Speed test of CPU and GPU versions of the function:
# print("CPU time:")
# cost_table = np.zeros((data.shape[0], data.shape[0]), dtype=np.int32)
# cost_table = calc_cost(data, popsize, vrp_capacity, cost_table)
# %timeit fitness(cost_table, individual)
# print("GPU time:")
# %timeit fitness_gpu[blocks,threads_per_block](cost_table_d, individual_d, zeroed_indiv_d, fitness_val_d)
###############################################################################################

## Calculate cost table:

### CPU version

In [ ]:
## Calculate cost table:
# @jit(nopython=True)
def calc_cost(data, popsize, vrp_capacity, cost_table):
    shifted_data = np.copy(data)
    for i in range(len(shifted_data[:,0])):
        shifted_data[i,0] = shifted_data[i,0] - 1

    for row in range(len(shifted_data[:,0])):
        for col in range(len(shifted_data[:,0])):
            cost_table[row, col] = round(hypot((shifted_data[row,2] - shifted_data[col,2]),\
                                                       (shifted_data[row,3] - shifted_data[col,3])))
    return cost_table

### GPU version

In [3]:
## Calculate cost table:
@cuda.jit
def calc_cost_gpu(data_d, popsize, vrp_capacity, cost_table_d):
    threadId_row, threadId_col = cuda.grid(2)
    
#     data_d[threadId_row,0] = data_d[threadId_row,0] - 1
    
####ceil() is used instead of round() as the latter crashes the kernel.
####This causes +1 values in some cost distances

    if (threadId_row <= data_d.shape[0]-1) and (threadId_col <= data_d.shape[0]-1):
        cost_table_d[threadId_row, threadId_col] = ceil(hypot(data_d[threadId_row,2] - data_d[threadId_col,2],\
                                                              data_d[threadId_row,3] - data_d[threadId_col,3]))
#     popArr = initializePop(data, popsize, vrp_capacity, cost_table)

### CPU version

In [ ]:
# @jit(nopython=True)
def adjust(individual, data, vrp_capacity, cost_table):

    # Delete duplicate nodes
    adjusted_indiv = np.zeros(individual.shape[0], dtype=np.int32)
    j = 0
    for i in range(len(individual)):
        if not np.any(individual[i] == adjusted_indiv):
            adjusted_indiv[j] = individual[i]
            j += 1

    # Delete ones and zeros
    adjusted_indiv = np.delete(adjusted_indiv, np.where(adjusted_indiv==1)[0])
    adjusted_indiv = np.delete(adjusted_indiv, np.where(adjusted_indiv==0)[0])

    
    # Insert missing nodes
    for i in range(data.shape[0]):
        if not np.any(data[i,0] == adjusted_indiv):
            adjusted_indiv = np.hstack((adjusted_indiv, np.array([data[i,0]], dtype=np.int32)))

    i = 0               # index
    reqcap = 0.0        # required capacity

    while i < len(adjusted_indiv): 
        if adjusted_indiv[i] != 1:
            reqcap += data[data[:,0] == adjusted_indiv[i]][0,1]
        else:
            reqcap = 0
        
        if reqcap > vrp_capacity: 
            adjusted_indiv = np.hstack((adjusted_indiv[:i], np.array([1], dtype=np.int32), adjusted_indiv[i:]))
            reqcap = 0.0
        i += 1
        
    if adjusted_indiv[0] != 1:
        adjusted_indiv = np.hstack((np.array([1], dtype=np.int32), adjusted_indiv))
    if adjusted_indiv[-1] != 1:
        adjusted_indiv = np.hstack((adjusted_indiv, np.array([1], dtype=np.int32)))
    
#     adjusted_indiv = np.hstack((adjusted_indiv, np.asarray([fitness(cost_table, adjusted_indiv)], dtype=np.int32)))
    return adjusted_indiv

### GPU version

In [ ]:
@cuda.jit
def adjust_gpu(data_d, vrp_capacity, cost_table_d, missing_d, pop_d, zeroed_indiv_d, fitness_val_d):
    
    # nodes represent the row/column index in the cost table
    threadId_row, threadId_col = cuda.grid(2)
    
    # Remove duplicated elements from every single individual/row in population array:
    r_flag = 9999 # A flag for removal/replacement
    
    if threadId_row <= pop_d.shape[0]-1 and threadId_col <= pop_d.shape[1]-1 and threadId_col != 0:
                    
        for i in range(threadId_col-1, -1, -1):
            if pop_d[threadId_row, threadId_col] == pop_d[threadId_row, i]\
            and pop_d[threadId_row, threadId_col] != 0:
                pop_d[threadId_row, threadId_col] = r_flag 
            
        for j in range(data_d.shape[0]):
            for i in range(threadId_col-1, -1, -1):
                if data_d[j,0] == pop_d[threadId_row, i]:
                    missing_d[threadId_row, j] = 0
                    break
                else:
                    missing_d[threadId_row, j] = data_d[j,0]
                     
    # Add missing nodes to every single individual:
            
    if threadId_col == pop_d.shape[1]-1:
        missing_elements = True
        for i in range(missing_d.shape[1]):
                if missing_d[threadId_row, i] != 0:
                    missing_elements = True
                    for j in range(pop_d.shape[1]):
                        if pop_d[threadId_row, j] == r_flag:
                            pop_d[threadId_row, j] = missing_d[threadId_row, i]
                            missing_d[threadId_row, i] = 0
                            break
                else:
                    missing_elements = False

        if not missing_elements:
        # shift individual's elements to the left for every inserted '1':
            for i in range(pop_d.shape[1], 0, -1):
                if pop_d[threadId_row, i] == r_flag:
                    for j in range(i, pop_d.shape[1]-1):
                        new_val = pop_d[threadId_row, j+1]
                        pop_d[threadId_row, j] = new_val

        reqcap = 0.0        # required capacity
        for i in range(pop_d.shape[1]-1):
            if pop_d[threadId_row, i] != 1 and pop_d[threadId_row, i] != 0:
                reqcap += data_d[pop_d[threadId_row, i]-1, 1]
                if reqcap > vrp_capacity:
                    # here will be the insert '1' algorithm:
                    new_val = 1
                    rep_val = pop_d[threadId_row, i]
                    
                    # shift individual's elements to the right for every inserted '1': 
                    for j in range(i, pop_d.shape[1]-1):
                        pop_d[threadId_row, j] = new_val
                        new_val = rep_val
                        rep_val = pop_d[threadId_row, j+1]
                    reqcap = 0.0                    
            else:
                reqcap = 0.0
                
            
#         # The last part is to add the individual's fitness value at the very end of it.
#             individual = pop_d[threadId_row,:]
#             fitness_gpu(cost_table_d, individual, zeroed_indiv_d, fitness_val_d)            
#             pop_d[threadId_row, -1] = fitness_val_d

## Evolve

### CPU version

In [ ]:
def evolvePop(pop, vrp_data, iterations, popsize, vrp_capacity, extended_cost, opt, cost_table=0):
    # Running the genetic algorithm
    run_time = timer()
    stucking_indicator = 0
    for i in tqdm(range(iterations)):
        old_best = pop[0][-1]
        nextPop = []
        nextPop_set = set()

        elite_count = len(pop)//20      
        sorted_pop = pop.copy()

        # Apply two-opt for the new top 5% individuals:
        for idx, individual in enumerate(sorted_pop[:elite_count]):
            if individual[0] >= i:
                sorted_pop[idx], cost = two_opt.two_opt(individual[1:-1], cost_table)
                sorted_pop[idx].append(9999)
                fitness_value = fitness(cost_table, sorted_pop[idx][:-1])
                sorted_pop[idx][-1] = (fitness_value)
                sorted_pop[idx].insert(0,individual[0])
        
        sorted_pop.sort(key= lambda elem: elem[-1])
        pop = sorted_pop.copy()
        
        start_evolution_timer = timer()
        # terminate if optimal is reached or runtime exceeds 1h
        if ((sorted_pop[0][-1] + extended_cost) > opt) and (timer() - run_time <= 60):
            nextPop = sorted_pop[:elite_count] # top 5% of the parents will remain in the new generation         

            # for j in range(round(((len(pop))-elite_count) / 2)):
            while len(nextPop_set) < popsize:
                # Selecting randomly 4 individuals to select 2 parents by a binary tournament
                parentIds = set()
                while len(parentIds) < 4:
                    parentIds.add(random.randint(0, len(pop) - 1))

                # Avoid stucking to a local minimum swap after 25 generations of identical fitness
                #if stucking_indicator >= 25:
                    #print('\nstucking is spotted', pop[1])
                    #for idx, swapped_indiv in enumerate(pop[1:elite_count]):
                        #i1 = swapped_indiv[1:round(len(swapped_indiv)/2)]
                        #i2 = swapped_indiv[round(len(swapped_indiv)/2): -1]
                        ## i1 = random.randint(1, len(swapped_indiv) - 2)
                        ## i2 = random.randint(1, len(swapped_indiv) - 2)
                        #swapped_indiv = i2
                        #swapped_indiv = np.append(swapped_indiv, i1)
                        ## swapped_indiv[i1], swapped_indiv[i2] = swapped_indiv[i2], swapped_indiv[i1]
                        #swapped_indiv = adjust(np.asarray(swapped_indiv[1:], dtype=np.float32), np.asarray(vrp_data, dtype=np.float32), vrp_capacity)
                        #fitness_val = fitness(np.asarray(vrp_data, np.float32), np.asarray(swapped_indiv[1:], np.float32))
                        ## swapped_indiv[-1] = fitness_val
                        #swapped_indiv = np.append(swapped_indiv, fitness_val)
                        #pop[idx] = swapped_indiv
                    #stucking_indicator = 0
               
                parentIds = list(parentIds)
                # Selecting 2 parents with the binary tournament
                parent1 = list(pop[parentIds[0]] if pop[parentIds[0]][len(pop[parentIds[0]])-1] < pop[parentIds[1]][len(pop[parentIds[1]])-1] else pop[parentIds[1]])
                parent2 = list(pop[parentIds[2]] if pop[parentIds[2]][len(pop[parentIds[2]])-1] < pop[parentIds[3]][len(pop[parentIds[3]])-1] else pop[parentIds[3]])

                child1 = parent1[1:].copy()
                child2 = parent2[1:].copy()

                # Performing Two-Point crossover and generating two children
                # Selecting (n/5 - 1) random cutting points for crossover, with the same points (indexes) for both parents, based on the shortest parent

                cutIdx = [0] * ((min(len(parent1) - 2, len(parent2) - 2))//5 - 1)
                for k in range(0, len(cutIdx)):
                    cutIdx[k] = random.randint(1, min(len(parent1) - 2, len(parent2) - 2))
                    while cutIdx[k] in cutIdx[:k]:
                        cutIdx[k] = random.randint(1, min(len(parent1) - 2, len(parent2) - 2))
                cutIdx.sort()
                for k in range(0, len(cutIdx), 2):
                    if len(cutIdx) %2 == 1 and k == len(cutIdx) - 1: # Odd number
                        child1[cutIdx[k]:] = child2[cutIdx[k]:]
                        child2[cutIdx[k]:] = child1[cutIdx[k]:]
                    else:                       
                        child1[cutIdx[k]:cutIdx[k + 1]] = child2[cutIdx[k]:cutIdx[k + 1]]
                        child2[cutIdx[k]:cutIdx[k + 1]] = child1[cutIdx[k]:cutIdx[k + 1]]        

                # Doing mutation: swapping two positions in one of the individuals, with 1:15 probability
                mutation_prob = 40
                if random.randint(1, mutation_prob) == 1:
                    # Random swap mutation
                    ptomutate = child1
                    i1 = random.randint(0, len(ptomutate) - 2)
                    i2 = random.randint(0, len(ptomutate) - 2)
                    # Repeat random selection if depot was selected
                    while ptomutate[i1] == 1:
                        i1 = random.randint(0, len(ptomutate) - 2)
                    while ptomutate[i2] == 1:
                        i2 = random.randint(0, len(ptomutate) - 2)
                    ptomutate[i1], ptomutate[i2] = ptomutate[i2], ptomutate[i1]

                if random.randint(1, mutation_prob) == 1:
                    ptomutate = child2
                    i1 = random.randint(0, len(ptomutate) - 2)
                    i2 = random.randint(0, len(ptomutate) - 2)
                    # Repeat random selection if depot was selected
                    while ptomutate[i1] == 1:
                        i1 = random.randint(0, len(ptomutate) - 2)
                    while ptomutate[i2] == 1:
                        i2 = random.randint(0, len(ptomutate) - 2)
                    ptomutate[i1], ptomutate[i2] = ptomutate[i2], ptomutate[i1]

                # Adjusting individuals               
                child1 = adjust(np.asarray(child1, dtype=np.float32), np.asarray(vrp_data, dtype=np.float32), vrp_capacity)
                child2 = adjust(np.asarray(child2, dtype=np.float32), np.asarray(vrp_data, dtype=np.float32), vrp_capacity)

                # # Apply 2-opt:
                # child1, fitness_val = two_opt.two_opt(child1[:-1], cost_table)
                # child2, fitness_val = two_opt.two_opt(child2[:-1], cost_table)

                fitness_val = fitness(cost_table, child1[:-1])
                child1[-1] = fitness_val
                
                fitness_val = fitness(cost_table, child2[:-1])
                child2[-1] = fitness_val

                child1 = list(child1)
                child2 = list(child2)

                child1.insert(0, i + 1)
                child2.insert(0, i + 1)

                # Add children to population iff they are better than parents
                if (child1[-1] < parent1[-1]) | (child1[-1] < parent2[-1]) | ((timer() - start_evolution_timer) > 30):
                    nextPop_set.add(tuple(child1))
                    # start_evolution_timer = timer()
                    # nextPop_set.add(tuple(parent1))
                
                if (child2[-1] < parent1[-1]) | (child2[-1] < parent2[-1]) | ((timer() - start_evolution_timer) > 30):
                    nextPop_set.add(tuple(child2))
                    # start_evolution_timer = timer()
                    # nextPop_set.add(tuple(parent2))   
                               
            nextPop = list(nextPop_set)

            # Updating population generation

            # random.shuffle(nextPop)
            nextPop = sorted(nextPop, key= lambda elem: elem[-1])

            if nextPop[0][-1] == old_best:
                stucking_indicator += 1
            else:
                stucking_indicator = 0

            pop = nextPop
            if not (i+1) % 5: # print population every 300 generations
                print(f'Population at generation {i+1}:{pop}\nBest: {pop[0][-1]}')
        elif (timer() - run_time >= 60):
            print('Time criteria is met')
            break
        elif (((sorted_pop[0][-1] + extended_cost) <= opt)):
            print('Cost criteria is met')
            break
    return (pop)

### GPU version

In [121]:
@cuda.jit(device=True)
def cut_points(pool_d, row, threadId_col):
    
    if threadId_col > 0 and threadId_col <= pool_d[row + 2, 2]:
        pool_d[row + 3, threadId_col] = 10 # should be replaced by a random number
        pool_d[row + 3, threadId_col+1] = 7
        pool_d[row + 3, threadId_col+2] = 5

# Crossover points (i.e., pool_d[row + 3,:]) must be ordered:
    if threadId_col == 1:       
        for i in range(1, pool_d[row + 2, 2]):
            min_val = pool_d[row + 3, i]
            min_index = i
            for j in range(i, pool_d[row + 2, 2]+1):
                if min_val > pool_d[row + 3, j]:
                    min_val = pool_d[row + 3, j]
                    min_index = j

            pool_d[row + 3, i], pool_d[row + 3, min_index] = pool_d[row + 3, min_index], pool_d[row + 3, i]
    cuda.syncthreads()
# ----------------------------------------------------------------------------------------------------------
@cuda.jit(device=True)
def swap_indices(pool_d, row, threadId_col, j):
    pool_d[row, threadId_col], pool_d[row+1, threadId_col] =\
    pool_d[row+1, threadId_col], pool_d[row, threadId_col]
# ----------------------------------------------------------------------------------------------------------
@cuda.jit(device=True)    
def cross_over(generations, popsize, opt, vrp_capacity, cost_table_d, cut_idx_d, pool_d, new_pop_d, pop_d, row):
    threadId_row, threadId_col = cuda.grid(2)
    
    if threadId_row < pop_d.shape[0]-1 and threadId_col <= pop_d.shape[1]-1 and threadId_col != 0:
    #   Create a pool of 4 randomly selected individuals:
        pool_d[row + 0, threadId_col] = pop_d[0, threadId_col] # 0 should be replaced by a random number
        pool_d[row + 1, threadId_col] = pop_d[1, threadId_col] # 1 should be replaced by a random number
        pool_d[row + 2, threadId_col] = pop_d[2, threadId_col] # 2 should be replaced by a random number
        pool_d[row + 3, threadId_col] = pop_d[3, threadId_col] # 3 should be replaced by a random number


    # Selecting 2 parents with the binary tournament
    # The first two pool_d rows are re-assigned as parents:
    # ----------------------------1st Parent--------------------------------------------------
        if pool_d[row, -1] < pool_d[row + 1, -1]:
            pool_d[row, threadId_col] = pool_d[row, threadId_col]
        else:
            pool_d[row, threadId_col] = pool_d[row + 1, threadId_col]

    # ----------------------------2nd Parent--------------------------------------------------
        if pool_d[row + 2, -1] < pool_d[row + 3, -1]:
            pool_d[row + 1, threadId_col] = pool_d[row + 2, threadId_col]
        else:
            pool_d[row + 1, threadId_col] = pool_d[row + 3, threadId_col]

        pool_d[row + 2, threadId_col] = 0
        pool_d[row + 3, threadId_col] = 0

    # Performing Two-Point crossover and generating two children:
    # Calculate the actual length of parents, put it in pool_d
        if pool_d[row, threadId_col] != 0:
            cuda.atomic.add(pool_d, (row + 2, pool_d.shape[1]-2), 1)

        if pool_d[row + 1, threadId_col] != 0:
            cuda.atomic.add(pool_d, (row + 3, pool_d.shape[1]-2), 1)

        pool_d[row + 2, 1] = \
        min(pool_d[row+2, pool_d.shape[1]-2], pool_d[row+3, pool_d.shape[1]-2]) # Minimum length of the two parents

    # Select (n/5 - 1) random cutting points for crossover based on the shortest parent
        pool_d[row + 2, 2] = pool_d[row + 2, 1]//5 - 1 # number of cutting points
        cut_points(pool_d, row, threadId_col)
# For odd swap indices, swap the chromosomes with indices less than the swap value:

#     if pool_d[row + 2, 2] %2 == 1: # Number of indices is odd

    if pool_d[row + 2, 2]%2 == 1:
        for j in range(1, (pool_d[row + 2, 2])+1):
            if threadId_col <= pool_d[row + 3, j] and threadId_col > pool_d[row + 3, j-1] and j%2 == 1:    
                swap_indices(pool_d, row, threadId_col, j)
    else:
        for j in range(1, (pool_d[row + 2, 2])+1):
            if (threadId_col <= pool_d[row + 3, j] and threadId_col > pool_d[row + 3, j-1] and j%2 == 1) or\
            (threadId_col > pool_d[row + 3, j] and j == pool_d[row + 2, 2]):    
                swap_indices(pool_d, row, threadId_col, j)
# ----------------------------------------------------------------------------------------------------------
@cuda.jit(device=True)
def mutate(pool_d, row, threadId_col):
# Mutation: swapping two positions in the children, with 1:40 probability
    mutation_prob = 40
#     if random.randint(1, mutation_prob) == 1:
    
    if threadId_col == 1:    
# Repeat random selection if depot was selected:    
        i1 = 1
        while pool_d[row, i1] == 1:
            i1 = 4
    #         i1 = random.randint(0, len(ptomutate) - 2)

        i2 = 1
        while pool_d[row, i2] == 1:
            i2 = 15
    #         i2 = random.randint(0, len(ptomutate) - 2)


        pool_d[row, i1], pool_d[row, i2] = pool_d[row, i2], pool_d[row, i1]

    # Repeat for the second child:    
        i1 = 1
        while pool_d[row+1, i1] == 1:
            i1 = 4
    #         i1 = random.randint(0, len(ptomutate) - 2)

        i2 = 1
        while pool_d[row+1, i2] == 1:
            i2 = 16
    #         i2 = random.randint(0, len(ptomutate) - 2)

        pool_d[row+1, i1], pool_d[row+1, i2] = pool_d[row+1, i2], pool_d[row+1, i1]
        cuda.syncthreads()
# ----------------------------------------------------------------------------------------------------------

@cuda.jit
def evolvePop_gpu(count, generations, popsize, opt, vrp_capacity, data_d, cost_table_d,\
                  cut_idx_d, pool_d, new_pop_d, pop_d, fitness_val_d):
    
    # nodes represent the row/column index in the cost table
    threadId_row, threadId_col = cuda.grid(2)
    stride = 4
    row = threadId_row*stride    
    
    if threadId_row < pop_d.shape[0]-1 and threadId_col <= pop_d.shape[1]-1 and threadId_col != 0:
        cross_over(generations, popsize, opt, vrp_capacity, cost_table_d, cut_idx_d, pool_d,\
                   new_pop_d, pop_d, row)
        mutate(pool_d, row, threadId_col)
        adjust_gpu(data_d, vrp_capacity, cost_table_d, missing_d, pop_d, \
                   zeroed_indiv_d, fitness_val_d, threadId_row, threadId_col)
        
        new_pop_d[threadId_row, 0] = count
        new_pop_d[threadId_row, threadId_col] = pool_d[row, threadId_col]

#     # Running the genetic algorithm
#     run_time = timer()
#     stucking_indicator = 0
#     for i in tqdm(range(iterations)):
#         old_best = pop[0][-1]
#         nextPop = []
#         nextPop_set = set()

#         elite_count = len(pop)//20      
#         sorted_pop = pop.copy()

# # Apply two-opt for the new top 5% individuals:
#         for idx, individual in enumerate(sorted_pop[:elite_count]):
#             if individual[0] >= i:
#                 sorted_pop[idx], cost = two_opt.two_opt(individual[1:-1], cost_table)
#                 sorted_pop[idx].append(9999)
#                 fitness_value = fitness(cost_table, sorted_pop[idx][:-1])
#                 sorted_pop[idx][-1] = (fitness_value)
#                 sorted_pop[idx].insert(0,individual[0])
        
#         sorted_pop.sort(key= lambda elem: elem[-1])
#         pop = sorted_pop.copy()
        
#         start_evolution_timer = timer()
# # terminate if optimal is reached or runtime exceeds 1h
#         if ((sorted_pop[0][-1] + extended_cost) > opt) and (timer() - run_time <= 60):
#             nextPop = sorted_pop[:elite_count] # top 5% of the parents will remain in the new generation         




# # Adjusting individuals               
#                 child1 = adjust(np.asarray(child1, dtype=np.float32), np.asarray(vrp_data, dtype=np.float32), vrp_capacity)
#                 child2 = adjust(np.asarray(child2, dtype=np.float32), np.asarray(vrp_data, dtype=np.float32), vrp_capacity)

#                 fitness_val = fitness(cost_table, child1[:-1])
#                 child1[-1] = fitness_val
                
#                 fitness_val = fitness(cost_table, child2[:-1])
#                 child2[-1] = fitness_val

#                 child1 = list(child1)
#                 child2 = list(child2)

# # Insert generation number at the beginning of every individual:
#                 child1.insert(0, i + 1)
#                 child2.insert(0, i + 1)

#                 # Add children to population iff they are better than parents
#                 if (child1[-1] < parent1[-1]) | (child1[-1] < parent2[-1]) | ((timer() - start_evolution_timer) > 30):
#                     nextPop_set.add(tuple(child1))
#                     # start_evolution_timer = timer()
#                     # nextPop_set.add(tuple(parent1))
                
#                 if (child2[-1] < parent1[-1]) | (child2[-1] < parent2[-1]) | ((timer() - start_evolution_timer) > 30):
#                     nextPop_set.add(tuple(child2))
#                     # start_evolution_timer = timer()
#                     # nextPop_set.add(tuple(parent2))   
                               
#             nextPop = list(nextPop_set)

#             # Updating population generation

#             # random.shuffle(nextPop)
#             nextPop = sorted(nextPop, key= lambda elem: elem[-1])

#             if nextPop[0][-1] == old_best:
#                 stucking_indicator += 1
#             else:
#                 stucking_indicator = 0

#             pop = nextPop
#             if not (i+1) % 5: # print population every 300 generations
#                 print(f'Population at generation {i+1}:{pop}\nBest: {pop[0][-1]}')
#         elif (timer() - run_time >= 60):
#             print('Time criteria is met')
#             break
#         elif (((sorted_pop[0][-1] + extended_cost) <= opt)):
#             print('Cost criteria is met')
#             break

In [122]:
new_pop_d = cuda.device_array_like(pop_d)
pool_d = np.zeros(shape=(5*popsize, pop_d.shape[1]), dtype=np.int32)
pool_d = cuda.to_device(pool_d)
cut_idx = [9999]*np.ones(shape=(pop_d.shape[1]), dtype=np.int32)
cut_idx_d = cuda.to_device(cut_idx)

count = 0

while count <= generations:
    evolvePop_gpu[blocks, threads_per_block]\
             (count, generations, popsize, opt, vrp_capacity, data_d, cost_table_d, \
              cut_idx_d, pool_d, new_pop_d, pop_d, fitness_val_d)
    cuda.synchronize()
    count += 1

import sys
np.set_printoptions(threshold=sys.maxsize)
print(pool_d.copy_to_host()[52:56,:])
print(new_pop_d.copy_to_host()[1:50,:])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<numba.cuda.compiler.DeviceFunctionTemplate object at 0x7f0a530b1cc0>) with argument(s) of type(s): (array(float32, 2d, A), float32, array(int32, 2d, A), readonly array(int32, 2d, A), array(int32, 2d, A), readonly array(int32, 1d, A), array(int32, 2d, A), int32, int32)
 * parameterized
In definition 0:
    TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function setitem>) with argument(s) of type(s): (readonly array(int32, 2d, A), (int32, int64), Literal[int](0))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals

## Adjust individuals:

In [117]:
@cuda.jit(device=True)
def add_missing_nodes(r_flag, threadId_row, threadId_col, missing_d, pop_d):
    # Add missing nodes to every single individual:
            
    if threadId_col == 1:
        missing_elements = True
        for i in range(missing_d.shape[1]):
            if missing_d[threadId_row, i] != 0:
                missing_elements = True
                for j in range(pop_d.shape[1]):
                    if pop_d[threadId_row, j] == r_flag:
                        pop_d[threadId_row, j] = missing_d[threadId_row, i]
                        missing_d[threadId_row, i] = 0
                        break
            else:
                missing_elements = False

#         if not missing_elements:
#         # shift individual's elements to the left for every inserted '1':
#             for i in range(pop_d.shape[1], 0, -1):
#                 if pop_d[threadId_row, i] == r_flag:
#                     for j in range(i, pop_d.shape[1]-1):
#                         new_val = pop_d[threadId_row, j+1]
#                         pop_d[threadId_row, j] = new_val
        cuda.syncthreads()
        
@cuda.jit(device=True)
def adjust_gpu(data_d, vrp_capacity, cost_table_d, missing_d,\
               pop_d, zeroed_indiv_d, fitness_val_d, threadId_row, threadId_col):
    
    # Remove duplicated elements from every single individual/row in population array:
    r_flag = 9999 # A flag for removal/replacement
    
    if threadId_row <= pop_d.shape[0]-1 and threadId_col <= pop_d.shape[1]-1 and threadId_col != 0:
                    
        for i in range(threadId_col-1, -1, -1):
            if pop_d[threadId_row, threadId_col] == pop_d[threadId_row, i]\
            and pop_d[threadId_row, threadId_col] != 0:
                pop_d[threadId_row, threadId_col] = r_flag 
            
        for j in range(data_d.shape[0]):
            for i in range(threadId_col-1, -1, -1):
                if data_d[j,0] == pop_d[threadId_row, i]:
                    missing_d[threadId_row, j] = 0
                    break
                else:
                    missing_d[threadId_row, j] = data_d[j,0]
        
        add_missing_nodes(r_flag, threadId_row, threadId_col, missing_d, pop_d)                     
        cuda.syncthreads()

#         reqcap = 0.0        # required capacity
#         for i in range(pop_d.shape[1]-1):
#             if pop_d[threadId_row, i] != 1 and pop_d[threadId_row, i] != 0:
#                 reqcap += data_d[pop_d[threadId_row, i]-1, 1]
#                 if reqcap > vrp_capacity:
#                     # here will be the insert '1' algorithm:
#                     new_val = 1
#                     rep_val = pop_d[threadId_row, i]
                    
#                     # shift individual's elements to the right for every inserted '1': 
#                     for j in range(i, pop_d.shape[1]-1):
#                         pop_d[threadId_row, j] = new_val
#                         new_val = rep_val
#                         rep_val = pop_d[threadId_row, j+1]
#                     reqcap = 0.0                    
#             else:
#                 reqcap = 0.0
                
# --------------------------------------------------------------------------------------------            
#         # The last part is to add the individual's fitness value at the very end of it.
#             individual = pop_d[threadId_row,:]
#             fitness_gpu(cost_table_d, individual, zeroed_indiv_d, fitness_val_d)            
#             pop_d[threadId_row, -1] = fitness_val_d

## Initialize population:

In [118]:
@cuda.jit
def initializePop_gpu(rng_states, data_d, vrp_capacity, cost_table_d, missing_d,\
                      pop_d, zeroed_indiv_d, fitness_val_d):
    
    threadId_row, threadId_col = cuda.grid(2)
    
    # Generate the individuals from the nodes in data_d:
    if threadId_col <= data_d.shape[0]-1:
        pop_d[threadId_row, threadId_col] = data_d[threadId_col, 0]
    
    # Randmly shuffle each individual on a separate thread:   
    col = 0
    if threadId_row <= pop_d.shape[0]-1 and threadId_col <= data_d.shape[0]-1 and threadId_col != 0:
        while col == 0:
            rnd = (xoroshiro128p_uniform_float32(rng_states, threadId_row*threadId_col)*(data_d.shape[0]-1))
            col = int(rnd)+1

        pop_d[threadId_row, threadId_col], pop_d[threadId_row, col] =\
        pop_d[threadId_row, col], pop_d[threadId_row, threadId_col]
        
        adjust_gpu(data_d, vrp_capacity, cost_table_d, missing_d, pop_d,\
                   zeroed_indiv_d, fitness_val_d, threadId_row, threadId_col)
        
    # Adjust individuals using adjust_gpu function:
    # Calculate fitness of each individual using fitness_gpu function:

# Main

In [120]:
vrp_capacity, data = readInput()
popsize = 100
generations = 7000
opt = 450

data_d = cuda.to_device(data)
cost_table_d = cuda.device_array(shape=(data.shape[0], data.shape[0]), dtype=np.int32)

pop = np.zeros((popsize, 2*data.shape[0]+2), dtype=np.int32)
pop_d = cuda.to_device(pop)

zeros = np.zeros(shape=(popsize, pop_d.shape[1]), dtype=np.int32)
missing_d = cuda.to_device(zeros)

zeroed_indiv = np.zeros(shape=(pop_d.shape[1]), dtype=np.int32)
zeroed_indiv_d = cuda.to_device(zeroed_indiv)
fitness_val = np.zeros(shape=(popsize,1), dtype=np.int32)
fitness_val_d = cuda.to_device(fitness_val)

# GPU grid configurations:
threads_per_block = (20, 20)
blocks_no = ceil(max(popsize, 2*data.shape[0]+2)/threads_per_block[0])

blocks = (blocks_no, blocks_no)
rng_states = create_xoroshiro128p_states(threads_per_block[0]**2  * blocks[0]**2, seed=1)
calc_cost_gpu[blocks, threads_per_block](data_d, popsize, vrp_capacity, cost_table_d)

initializePop_gpu[blocks, threads_per_block]\
                 (rng_states, data_d, vrp_capacity, cost_table_d, missing_d,\
                  pop_d, zeroed_indiv_d, fitness_val_d)

np.set_printoptions(threshold=sys.maxsize)
print(pop_d.copy_to_host()[0:5,:], end='\n-----------------------\n')
print(missing_d.copy_to_host()[0:5,:], end='\n')

# fitness_gpu[blocks,threads_per_block](cost_table_d, adjusted_indiv, zeroed_indiv_d, fitness_val_d)
# print(fitness_val_d.copy_to_host()[0])
# print(cost_table_d.copy_to_host())
###############################################################################################
# Speed test of CPU and GPU versions of the function:
# cost_table = np.zeros((data.shape[0],data.shape[0]), dtype=np.int32)
# print(calc_cost(data, popsize, vrp_capacity, cost_table).shape)
# print('CPU time:')
# %timeit calc_cost(data, popsize, vrp_capacity, cost_table)
# print('GPU time:')
#%timeit calc_cost_gpu[blocks, threads_per_block](data_d, popsize, vrp_capacity, cost_table_d)
################################################################################################

Reading data file... Done.
[[ 1  3 13  1  2  2  3 16  4  5  6  7  5  8  9 10  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 1 15  1 14 11 10  5  4  3  6  2  3  4 16 12  9  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 1 12  7  9 13  8  3 16  5  2  1 15  2  6  3  4  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 1 16  5  6  3  4 12  1  2 14  8 15 13  2  3  4  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 1 14  4 15 13 16 12  1  9  2  2 10  3 11  3  4  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]]
-----------------------
[[ 0  0  0  4  0  6  7  8  0  0 11 12  0 14 15  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  7  8  9  0  0 12 13  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  6  0  0  0 10 11  0  0 14  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  7  0  9 10 11  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  

In [ ]:


# #%timeit adjust_gpu[blocks,threads_per_block]\
#  #(data_d, vrp_capacity, cost_table_d, missing_d, pop_d)
# adjust_gpu[blocks, threads_per_block]\
# (data_d, vrp_capacity, cost_table_d, pop_d, zeroed_indiv_d, fitness_val_d)

# # print(missing_d.copy_to_host()[50:65,:])
# print(pop_d.copy_to_host()[0:4,:])